# Exercise 5 - Image analysis and compression

In [ ]:
#loading a few python modules used for image processing, io and plotting

import numpy as np
import numpy.matlib
from matplotlib import pyplot as plt
from scipy import ndimage
from scipy import misc
from scipy import fftpack
import skimage
import imageio

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from accum import accum

from skimage.transform import resize
# for 16 bit PNG support in imageio if needed, imageio.plugins.freeimage.download() might need to be run once per system (or exr/hdr/pgm/ppm support)

In [ ]:
# Load our favorite test image
peppersImg = imageio.imread('peppers.png')
peppersImg = np.float64( peppersImg ) / 255

# Create standard 75% color bars with the width of the peppers image and the following height:
bars_and_gradient_height = 64

colorBar75vals = np.array([[[0, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 1], [0, 1, 0], [0, 1, 1], [1, 1, 0], [1, 1, 1]]]) * 0.75
colorBar75 = np.tile( colorBar75vals, (bars_and_gradient_height*np.int32(np.size(peppersImg,1)/np.size(colorBar75vals,1)), 1, 1) )
colorBar75 = np.reshape( colorBar75, (64,np.size(peppersImg,1),3), order='F' ) #R colorBar75 = # Your code here

# Create a gradient from 0...1 with the width of the peppers image and the same height as the color bars
gradient = np.tile( np.reshape( np.linspace(0,1,np.size(peppersImg,1)), (1,np.size(peppersImg,1)) )[:,:,np.newaxis], (bars_and_gradient_height, 1, 3) ) #R gradient = # Your code here

# Debug shapes to see if they fit
print( peppersImg.shape )
print( colorBar75.shape )
print( gradient.shape )

In [ ]:
# Concatenate and show Image
plt.subplot(1,2,1)
img = np.concatenate( ( peppersImg, colorBar75, gradient), axis=0 )
plt.imshow( img );
plt.title("Your Image")
plt.subplot(1,2,2)
plt.imshow( plt.imread( "exercise_05_results_for_reference/peppers_bar_gradient.png" ), interpolation = 'catrom' );
plt.title("Reference Image\nwith color bars\nand gradient");

## 5.1 Histogram

Histograms are useful to see the distribution of code values between blacks, mid tones and highlights and to eliminate color casts. They can also be used to detect reduced quantization.

In [ ]:
# Finish the function `calculateHistogramm` by looping over all bins and not using the Python histogram functions.
def calculateHistogramm(img, bins = 256, minval = 0.0, maxval = 1.0):
    data = img.copy()#R # Your code here
    #R # Your code here
    data = (data - minval) / (maxval - minval)#R # Your code here
    data[data < 0.0] = -1000#R # Your code here
    data[data > 1.0] = -1000#R # Your code here
    data = np.floor(data * bins)#R # Your code here
#R # Your code here
    result = np.zeros((bins))#R # Your code here
    for i in range(bins):#R # Your code here
        result[i] = np.sum(data == i)#R # Your code here
#R # Your code here
    return result
    # The result vector is a 1D-vector with the siye of the number of bins containing the number of pixel values that fall into this bin.

In [ ]:
# Apply the ASC-CDL grading controls from 4.2 to get an intuition how the histogram behaves.
#R # Your code here
#R # Your code here
#R # Your code here
#R # Your code here
gradedImg = img #R gradedImg = # Your code here

pepper_hist = calculateHistogramm( gradedImg )
hist_range  = np.arange(0, pepper_hist.shape[0])

plt.figure(figsize=(20,5))
plt.subplot(2,2,(1,3))
plt.imshow( gradedImg )
plt.subplot(2,2,2)
img = np.concatenate( ( peppersImg, colorBar75, gradient), axis=0 )
plt.bar(hist_range, pepper_hist);
plt.title("Your Histogram")
plt.subplot(2,2,4)
plt.imshow( plt.imread( "exercise_05_results_for_reference/histogram.png" ), interpolation = 'catrom' );
plt.axis('off')
plt.title("Reference Histogram if no grading is applied");

In [ ]:

# Play with the histogram resolution to detect that peppers.png seems to be 8 bits.
# Increasing the number of bins is one way to do this, but setting minval and maxval to appropriate numbers may be more clever. 
n_bins = 512#R # Your code here
hist_range  = np.arange(0, n_bins)#R # Your code here
f = plt.figure(figsize=(20, 6)) #R # Your code here
a = f.subplots() #R # Your code here
color = ['red', 'green', 'blue'] #R # Your code here
for i in range(3): #R # Your code here
    pepper_hist = calculateHistogramm(img[..., i], bins=n_bins) #R # Your code here
    a.bar(hist_range, pepper_hist, color=color[i]) #R # Your code here
plt.show() #R # Your code here


In [ ]:
# Try to hide the fact that peppers.png is only 8bits by resizing it.
# What may be a better choice than resizing? Would this solve visible quantization artifacts in the image?
pepper_hist = calculateHistogramm(img[:,:,2], minval=0.3, maxval=0.5) #R # Your code here
hist_range  = np.arange(0, pepper_hist.shape[0]) #R # Your code here
plt.figure(figsize=(15,5)) #R # Your code here
plt.bar(hist_range, pepper_hist); #R # Your code here

img_scaled = resize(img, (520,520))

pepper_hist = calculateHistogramm(img_scaled[:,:,2], minval=0.3, maxval=0.5 ) #R # Your code here
hist_range  = np.arange(0, pepper_hist.shape[0]) #R # Your code here
fig = plt.figure(figsize=(15,5)) #R # Your code here
ax = fig.subplots() #R # Your code here
ax.bar(hist_range, pepper_hist) #R # Your code here
ax.set_yscale('log') #R # Your code here

img.shape

In [ ]:
pepper_hist = calculateHistogramm(img[:,:,2], minval=0.3, maxval=0.4 ) #R # Your code here
hist_range  = np.arange(0, pepper_hist.shape[0]) #R # Your code here
plt.figure(figsize=(15,5)) #R # Your code here
plt.bar(hist_range, pepper_hist); #R # Your code here

img_noise = img + np.random.randn( img.shape[0], img.shape[1], img.shape[2] ) / 255

pepper_hist = calculateHistogramm(img_noise[:,:,2], minval=0.3, maxval=0.4 ) #R # Your code here
hist_range  = np.arange(0, pepper_hist.shape[0]) #R # Your code here
fig = plt.figure(figsize=(15,5)) #R # Your code here
ax = fig.subplots() #R # Your code here
ax.bar(hist_range, pepper_hist) #R # Your code here
ax.set_yscale('log') #R # Your code here

img.shape

## 5.2 Waveform

A waveform monitor can be thought as being histogram per column, but the height of the histogram bar is now shown as pixel intensity. 

In [ ]:
# Finish the function `calculateWaveform`.
def calculateWaveform(img, bins = 256, luma = False, minval = 0.0, maxval = 1.0):
    if not (luma and img.ndim == 3) and not (not luma and img.ndim >= 2 and img.ndim <= 3):
        raise Exception('Invalid input dimensions or luma conversion requested on 2D image')

    data = img.copy()
    if luma and len(data.shape) == 3:
        data = 0.2126 * img[..., 0] + 0.7152 * img[..., 1] + 0.0722 * img[..., 2]
    luma = data.ndim == 2 # in case a 2D image has been imported
    
    data = (data - minval) / (maxval - minval)
    data[data < 0.0] = 0.0
    data[data > 1.0] = 1.0
    data = np.floor(data * (bins - 0.000001))

    dimensions = (bins, data.shape[1])
    if not luma:
        dimensions = (dimensions[0], dimensions[1], data.shape[2])

    result = np.zeros(dimensions)
    for i in range(bins):
        result[i, ...] = np.sum(data == i, axis=0)
    result = np.log(result + 1) / np.log(bins)

    return np.flip(result, 0)

In [ ]:
# Apply the grading controls from 4.2 and get an intuition how the waveform behaves.
#R # Your code here
#R # Your code here
#R # Your code here
#R # Your code here
gradedImg = img #R gradedImg = # Your code here

wf = calculateWaveform(gradedImg, bins=256)
plt.subplot(1,2,1)
plt.imshow( np.concatenate( ( gradedImg, wf), axis=0 ) );
plt.title("Your image");
plt.subplot(1,2,2)
plt.imshow( plt.imread( "exercise_05_results_for_reference/img_and_waveform.png" ), interpolation = 'catrom' );
plt.axis('off')
plt.title("Reference Image and Waveform\nif no grading is applied");

In [ ]:
# Change the bins to 512. How can you detect limited tonal resolution using a waveform? Did you already observe this in grading?
wf = calculateWaveform(gradedImg, bins=512)
# plt.figure(figsize=(10,20)) # You may need to enlarge the plot to see the effect...
plt.imshow( np.concatenate( ( gradedImg, wf), axis=0 ) );

## 5.3 Vectorscope
Colorist often use the vectorscope to adjust color balance. 

In [ ]:
# Finish the function `calculateVectorscope`.
# Hint: For calculating the 2D histogram ‘accum’ will be much faster compared to two loops
def calculateVectorscope(img, bins=256):
    if img.ndim < 2 or img.shape[-1] != 3:
        raise Exception('Invalid input size')

    RGB2YCbCr709Mx = np.array([[0.2126, -0.114572, 0.5], [0.7152, -0.385428, -0.454153], [0.0722, 0.5, -0.045847]])
    data = img.reshape(-1, 3)
    data = np.matmul(data, RGB2YCbCr709Mx)

    data = data + 0.5
    data[data < 0.0] = 0.0
    data[data > 1.0] = 1.0
    data = np.uint32(np.floor(data * (bins - 0.000001)))

    result = accum(data[:,1:], np.ones((data.shape[0])), size=(bins, bins))
    result = np.swapaxes(result, 0, 1)
    return np.flip(result, axis=0)

In [ ]:
# Add some noise to the image before feeding it into the vectorscope so that the color bars do not only end up at the same pixel position.
noise_img = img + (np.random.randn(img.shape[0], img.shape[1], 3) - 0.5) / 256

vectorscopeOutput = calculateVectorscope( noise_img )

plt.subplot(1,2,1)
plt.imshow(np.log2(vectorscopeOutput + 1) / np.log2(np.amax( vectorscopeOutput ) + 1), cmap='gray', vmin=0, vmax=1.0)
plt.title("Your Vectorscope");
plt.subplot(1,2,2)
plt.imshow( plt.imread( "exercise_05_results_for_reference/vectorscope.png" ), interpolation = 'catrom' );
plt.axis('off')
plt.title("Reference Vectorscope\nif no grading is applied");

In [ ]:
# Apply the grading controls from 4.2 and get an intuition how the vectorscope behaves. Especially try saturation
#R # Your code here
#R # Your code here
#R # Your code here
#R # Your code here
gradedImg = img #R gradedImg = # Your code here
#R gradedImg = # Your code here
foo = calculateVectorscope(noise_img)#R gradedImg = # Your code here
plt.figure(figsize=(15,10))#R gradedImg = # Your code here
plt.imshow(np.log2(foo + 1) / np.log2(np.amax( foo ) + 1), cmap='gray', vmin=0, vmax=1.0)#R gradedImg = # Your code here

## 5.4 Decorelation Color Space/perception

The lower spatial resolution of the human visual system for color compared to achromatic structures can be exploited by storing color with less spatial resolution compared to luminance. This process is called color subsampling.

**How to transform R’G’B’ signals to be able to apply color subsampling? Luma and chroma need to be separated!**

* Transform the image `ct_kiste.jpg` from sRGB to Rec.709 Y’CbCr and back to sRGB. Hint: Search for the original Rec.709 specification https://www.itu.int/rec/R-REC-BT.709-6-201506-I/en.
* Apply a Gaussian blur filter with different radius in Y’ in the Rec.709 Y’CbCr domain.
* Apply a Gaussian blur filter with different radius in Cb and Cr in the Rec.709 Y’CbCr domain.
* What maximum Gaussian blur radius is acceptable in Y’ what maximum radius is acceptable in CbCr?
* Give at least two examples why subsampling should only be used in image distribution but not in image acquisition and contribution.

In [ ]:
def applyColorMatrix(values, mat):
    tmp = values.reshape(-1, 3)
    tmp = np.matmul(tmp, mat)
    return np.reshape(tmp, values.shape)

In [ ]:
RGB2YCbCr709Mx = np.array([[0.2126, -0.114572, 0.5], [0.7152, -0.385428, -0.454153], [0.0722, 0.5, -0.045847]])
YCbCr2RGB709Mx = np.linalg.inv(RGB2YCbCr709Mx)

In [ ]:
ycbcr = applyColorMatrix(img, RGB2YCbCr709Mx)

ycbcr[:,:,0] = ndimage.gaussian_filter(ycbcr[:,:,0], 0.2)
ycbcr[:,:,1] = ndimage.gaussian_filter(ycbcr[:,:,1], 2.5)
ycbcr[:,:,2] = ndimage.gaussian_filter(ycbcr[:,:,2], 2.5)

plt.figure(figsize=(15, 20))
plt.imshow(applyColorMatrix(ycbcr, YCbCr2RGB709Mx));

## 5.5 Image compression

How can we reduce image file size without reducing spatial or tonal resolution? By reducing quantization in frequency domain instead of spatial domain.
* Read peppers.png and convert to 0…1 domain.
* Calculate luma for the peppers image and plot this image.
* Convert each 8x8 block of this image to frequency domain using the MATLAB functions ‘blockproc’ and ‘dct2’.
* Plot the image in frequency domain.
* Convert back to spatial domain using blockproc again and idct2.
* Make sure there are no roundtrip errors by subtracting the original image from the reconstructed image. The mean absolute difference should be below 10-14.
* We will use the JPEG quantization matrix as supplied in the .m file and a QP of 2.7 to reduce the energy of the high frequency DCT coefficients. Perform an elementwise multiplication of each block of the image in frequency domain by the quantization matrix.
* The supplied code helps you to find the needed maximum bits needed to encode each block position for this image. You should end up with a number below 64 bits per block.
* Now let’s decode again. First multiply each block position by the quantization matrix to rescale these values again.
* Convert back from frequency domain to spatial domain.
* Plot original image, DCT image and reconstructed image.
* Alternatively quantize the original image to 1bit per pixel get down to 64 bits per block.
* Scale the image to half the horizontal and vertical size to be able to spend 4 bits per pixel.
* Scale the image to the exact size so that each pixel can use 8bits, but the needed file size stays the same as for the images that only used 64 bits per block.

In [ ]:
luma = applyColorMatrix(img, RGB2YCbCr709Mx)[:,:, 0]

In [ ]:
def odct2(a):
    return fftpack.dct(fftpack.dct(a, axis=0, norm='ortho' ), axis=1, norm='ortho')
def idct2(a):
    return fftpack.idct(fftpack.idct(a, axis=0 , norm='ortho'), axis=1 , norm='ortho')

def blockProc(data, blockfunc):
    result = np.zeros(data.shape)

    #for y in range(data.shape[0]//8):  ## Umbauen auf range(0, dct_image.shape[0], 8)
    #    for x in range(data.shape[1]//8):
    for y in range(0, data.shape[0], 8):
        for x in range(0, data.shape[1], 8):
            block = data[y:y+8, x:x+8]
            block = blockfunc(block)
            result[y:y+8, x:x+8] = block
    
    return result

In [ ]:
dct_img = blockProc(luma, odct2)
plt.figure(figsize=(15, 15))
plt.imshow(dct_img, cmap='gray');

In [ ]:
idct_img = blockProc(dct_img, idct2)
plt.figure(figsize=(15, 15))
plt.imshow(dct_img, cmap='gray');

In [ ]:
diff = luma - idct_img
print(np.amin(diff), np.amax(diff))

In [ ]:
qMatrix = np.array([[16,  11,  10,  16,  24,   40,   51,   61],
                    [12,  12,  14,  19,  26,   58,   60,   55],
                    [14,  13,  16,  24,  40,   57,   69,   56],
                    [14,  17,  22,  29,  51,   87,   80,   62],
                    [18,  22,  37,  56,  68,  109,  103,   77],
                    [24,  35,  55,  64,  81,  104,  113,   92],
                    [49,  64,  78,  87, 103,  121,  120,  101],
                    [72,  92,  95,  98, 112,  100,  103,   99]])
qp = 2.7
qmat_q = qMatrix * qp
qmat_q

In [ ]:
dct_qm = blockProc(luma, lambda x : np.divide(odct2(x), qmat_q))
plt.figure(figsize=(15, 15))
plt.imshow(dct_qm, cmap='gray');

In [ ]:
# round and quantize
dct_rounded = np.round(dct_qm * 255) / 255

idct_qm = blockProc(dct_rounded, lambda x : idct2(np.multiply(x, qmat_q)))
plt.figure(figsize=(15, 15))
plt.imshow(idct_qm, cmap='gray');

In [ ]:
diff = idct_qm - luma
print(np.amin(diff), np.amax(diff))

In [ ]:
import numpy as np
import imageio
from matplotlib import pyplot as plt

checkers = np.zeros((320,240), dtype=np.uint8)
checkers[::2,::2] = 255
checkers[1::2,1::2] = 255

plt.imshow(checkers, cmap='gray');